# Databricksにおける心電図(ECG)データ処理のためのデータのステージング

このデモでは、PTB Diagnostic [ECG Database](https://physionet.org/physiobank/database/ptbdb/)からのデータを用いて、ECGデータをDelta Lakeへ格納する際にDatabricksレイクハウスプラットフォームが提供する機能をご説明します。これは4つあるノートブックの1つめとなります。このノートブックでは、PTBの研究で収集された人々のECGデータを示す[WFDB](https://wfdb.readthedocs.io/en/latest/)ファイルをダウンロードします。そして、データに対する機械学習、ストリーミング分析を行えるように、このデータを[Delta Lake](https://delta.io/)にロードします。

最初に、AWS S3、Azure Blob Storage上に乗るメタデータ管理レイヤーの[DBFS](https://qiita.com/taka_yayoi/items/e16c7272a7feb5ec9a92)にデータをダウンロードします。マジックコマンド`%sh`を用いて、[Databricksノートブック上でシェルコマンドを実行](https://qiita.com/taka_yayoi/items/dfb53f63aed2fbd344fc#%E6%B7%B7%E6%88%90%E8%A8%80%E8%AA%9E)することができます。Physionetのウェブサイトからデータをダウンロードするためにシェルで`wget`コマンドを使用します。

**参考資料**
- [Delta Lake、Keras、MLflowを用いた機械学習による医療機器データのモニタリング \- Qiita](https://qiita.com/taka_yayoi/items/65e463a3eab84d4e2ce7)
- [Monitoring patient medical device data with ML \+ Delta Lake, Keras, and MLflow](https://databricks.com/blog/2019/09/12/monitor-medical-device-data-with-machine-learning-using-delta-lake-keras-and-mlflow-on-demand-webinar-and-faqs-now-available.html)

<table>
  <tr><th>作者</th><th>Databricks Japan</th></tr>
  <tr><td>日付</td><td>2021/7/9</td></tr>
  <tr><td>バージョン</td><td>1.0</td></tr>
  <tr><td>クラスター</td><td>8.3ML</td></tr>
</table>
<img style="margin-top:25px;" src="https://jixjiadatabricks.blob.core.windows.net/images/databricks-logo-small-new.png" width="140">

## データのダウンロード

In [0]:
%sh

wget https://physionet.org/static/published-projects/ptbdb/ptb-diagnostic-ecg-database-1.0.0.zip

unzip ptb-diagnostic-ecg-database-1.0.0.zip

--2021-07-08 04:13:54-- https://physionet.org/static/published-projects/ptbdb/ptb-diagnostic-ecg-database-1.0.0.zip
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1790456359 (1.7G) [application/zip]
Saving to: ‘ptb-diagnostic-ecg-database-1.0.0.zip’

 0K .......... .......... .......... .......... .......... 0% 752K 38m44s
 50K .......... .......... .......... .......... .......... 0% 764K 38m26s
 100K .......... .......... .......... .......... .......... 0% 768K 38m16s
 150K .......... .......... .......... .......... .......... 0% 765K 38m13s
 200K .......... .......... .......... .......... .......... 0% 1.08M 35m52s
 250K .......... .......... .......... .......... .......... 0% 745K 36m24s
 300K .......... .......... .......... .......... .......... 0% 2.06M 33m11s
 350K .......... .......... .......... .......... .......... 0% 759K 33m50s
 400K .......... .......... .......... .......... .......... 0% 24.4M 30m12s
 450K .......... .......... .......... .......... .......... 0% 785K 30m53s
 500K .......... .......... .......... .......... .......... 0% 10.9M 28m19s
 550K .......... .......... .......... .......... .......... 0% 23.9M 26m3s
 600K .......... .......... .......... .......... .......... 0% 30.4M 24m7s
 650K .......... .......... .......... .......... .......... 0% 18.4M 22m31s
 700K .......... .......... .......... .......... .......... 0% 913K 23m8s
 750K .......... .......... .......... .......... .......... 0% 29.2M 21m45s
 800K .......... .......... .......... .......... .......... 0% 26.7M 20m32s
 850K .......... .......... .......... .......... .......... 0% 18.6M 19m29s
 900K .......... .......... .......... .......... .......... 0% 22.4M 18m31s
 950K .......... .......... .......... .......... .......... 0% 16.3M 17m41s
 1000K .......... .......... .......... .......... .......... 0% 25.0M 16m53s
 1050K .......... .......... .......... .......... .......... 0% 66.5M 16m8s
 1100K .......... .......... .......... .......... .......... 0% 149M 15m27s
 1150K .......... .......... .......... .......... .......... 0% 1.52M 15m35s
 1200K .......... .......... .......... .......... .......... 0% 27.6M 15m0s
 1250K .......... .......... .......... .......... .......... 0% 2.55M 14m51s
 1300K .......... .......... .......... .......... .......... 0% 24.1M 14m21s
 1350K .......... .......... .......... .......... .......... 0% 23.2M 13m53s
 1400K .......... .......... .......... .......... .......... 0% 43.7M 13m25s
 1450K .......... .......... .......... .......... .......... 0% 32.5M 13m0s
 1500K .......... .......... .......... .......... .......... 0% 24.4M 12m37s
 1550K .......... .......... .......... .......... .......... 0% 35.3M 12m15s
 1600K .......... .......... .......... .......... .......... 0% 69.3M 11m53s
 1650K .......... .......... .......... .......... .......... 0% 78.1M 11m33s
 1700K .......... .......... .......... .......... .......... 0% 66.7M 11m14s
 1750K .......... .......... .......... .......... .......... 0% 41.5M 10m56s
 1800K .......... .......... .......... .......... .......... 0% 30.2M 10m40s
 1850K .......... .......... .......... .......... .......... 0% 36.7M 10m24s
 1900K .......... .......... .......... .......... .......... 0% 30.2M 10m10s
 1950K .......... .......... .......... .......... .......... 0% 22.7M 9m57s
 2000K .......... .......... .......... .......... .......... 0% 29.9M 9m43s
 2050K .......... .......... .......... .......... .......... 0% 29.5M 9m31s
 2100K .......... .......... .......... .......... .......... 0% 28.1M 9m19s
 2150K .......... .......... .......... .......... .......... 0% 27.1M 9m8s
 2200K .......... .......... .......... .......... .......... 0% 2.21M 9m13s
 2250K .......... .......... .......... .......... .......... 0% 6.66M 9m6s
 2300K .......... .......... .......... .......... .......... 0% 2.56M

## DBFSへのデータコピー

データのダウンロードが完了したら、マジックコマンド`%fs`を用いて、クラスターのドライバーノードのローカルファイルシステムからDBFSにデータをコピーします。これは[ファイルシステムオペレーションのためのDatabricksユーティリティ](https://docs.databricks.com/user-guide/databricks-file-system.html#dbutils)へのショートカットとなります。

**注意** 以下のコピー先のパス`/tmp/takaaki.yayoi@databricks.com...`は適宜変更してください。

In [0]:
%fs cp -r file:///databricks/driver/ptb-diagnostic-ecg-database-1.0.0 /tmp/takaaki.yayoi@databricks.com/hls/ptb-diagnostic-ecg

res0: Boolean = true

## 必要なライブラリのインストール

分析を行う前に必要なライブラリをインストールする必要があります。ここでは、CondaとMLランタイムを使用しており、ライブラリを永続的にインストールする簡単な方法は、[init scripts](https://docs.databricks.com/user-guide/clusters/conda.html#id10)を用いることです。以下のセルを実行してスクリプトをDBFSに書き出した後で、このスクリプトを[クラスタースコープinit script](https://docs.databricks.com/user-guide/clusters/init-scripts.html#cluster-scoped-init-scripts)としてアタッチし、クラスターを再起動する必要があります。

**注意** 
- 以下のセルの実行は初回のみとなります。作成されたinit scriptをクラスターに設定してください。
- `/tmp/takaaki.yayoi@databricks.com...`のパスは適宜変更してください。

In [0]:
dbutils.fs.rm("/tmp/takaaki.yayoi@databricks.com/hls/wfdb.sh")
dbutils.fs.put("/tmp/takaaki.yayoi@databricks.com/hls/wfdb.sh", '''#!/bin/bash
set -ex
/databricks/python/bin/python -V
. /databricks/conda/etc/profile.d/conda.sh
conda activate /databricks/python
conda install -c conda-forge -y wfdb mlflow tqdm''')

Wrote 178 bytes.
Out[2]: True

## UDF(ユーザー定義関数)を用いてWFDBファイルをDelta Lakeにロードする

PhysionetのWFDBファイルをDelta Lakeにロードするには、変換するためのコードを記述する必要があります。Apache SparkにはビルトインのWFDBローダーがないため、WFDBレコードをSparkデータフレームにパースするためのカスタムコードを記述する必要があります。このためには、指定されたファイルパスからWFDBレコードを読み込み、ファイルごとにSparkデータフレームのレコードを生成する[ユーザー定義関数](https://docs.databricks.com/spark/latest/spark-sql/udf-python.html)を定義します。

最初のステップとして、UDFのために必要なライブラリをインポートします。

In [0]:
from wfdb import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import time

次にDBFS上に格納したファイルパス、および書き出し先のパスを指定します。

**注意**
`/tmp/takaaki.yayoi@databricks.com...`のパスは適宜変更してください。

In [0]:
LOCAL_DATA_PATH = '/tmp/takaaki.yayoi@databricks.com/hls/ptb-diagnostic-ecg'
DELTA_PATH = '/tmp/takaaki.yayoi@databricks.com/hls/ecg/staged/'

### UDFの定義及び登録

これでUDFを記述できるようになりました。このUDFでは、クラウドストレージにアクセスするために[Databricks FUSE mount](https://docs.databricks.com/applications/deep-learning/data-prep/ddl-storage.html)を用いて、標準的なPOSIXファイルAPIを介してWFDBにアクセスします。これによって、変更を加えることなしにwfdbライブラリを利用できます。我々のUDFはPythonのdictを返却し、Sparkはこれをrowにエンコードします。それぞれのWFDBファイルに対して、患者のID、ファイルに添付されたコメント、ファイルでエンコードされたすべてのシグナルからなるrowを生成します。

In [0]:
def extract_signals(record_name):
  
  pname = record_name.split('/')[0]
  
  record_id = record_name.replace('/','_')
  record_path = '/dbfs'+LOCAL_DATA_PATH+'/'+record_name
  record = io.rdrecord(record_name=record_path)
  record_comments = {m.split(':')[0]:m.split(':')[1] for m in record.comments}
  record_sig_name = record.sig_name
  record_signals = {}
  
  for i in range(len(record_sig_name)):
    record_signals[record_sig_name[i]] = (record.p_signal[:,i]).tolist()
    
  return {'patient_id': pname, 'comments': record_comments, 'signals': record_signals}

関数を記述したら、Sparkに関数を登録します。このためには、関数から返却されるスキーマを定義する必要があります。

In [0]:
from pyspark.sql.types import MapType, StringType, IntegerType, StructType, StructField, ArrayType, FloatType
from pyspark.sql import functions as F

udf_schema = StructType([ 
  StructField("patient_id", StringType(), True),
  StructField("comments", MapType(StringType(), StringType()), True),
  StructField("signals", MapType(StringType(), ArrayType(FloatType())), True),
])

extract_signals_udf = F.udf(extract_signals, udf_schema)

## WFDBファイルをSparkデータフレームにロード

ファイルパスを受け取り、患者に関するシグナル、メタデータからなるSparkデータフレームの行を生成するUDFを登録しました。ファイルをロードするには、最初にWFDBファイルが格納されているパスの一覧を作成する必要があります。ここでは、事前に準備したサンプルIDのリストを用いますが、同じことを[dbutils.fs APIs](https://docs.databricks.com/user-guide/dev-tools/dbutils.html#file-system-utilities)で行えます。

In [0]:
from pyspark.sql import Row
record_names = io.get_record_list('ptbdb')
df_record_names = spark.createDataFrame(list(map(lambda x: Row(record_name=x,record_id=x.replace('/','-')), record_names)))
display(df_record_names)

record_name,record_id
patient001/s0010_re,patient001-s0010_re
patient001/s0014lre,patient001-s0014lre
patient001/s0016lre,patient001-s0016lre
patient002/s0015lre,patient002-s0015lre
patient003/s0017lre,patient003-s0017lre
patient004/s0020are,patient004-s0020are
patient004/s0020bre,patient004-s0020bre
patient005/s0021are,patient005-s0021are
patient005/s0021bre,patient005-s0021bre
patient005/s0025lre,patient005-s0025lre


これでデータをロードする準備が整いました。最初に並列性を高めるためにデータフレームを再パーティションします。そして、事前定義SQL関数のように、Spark SQLのselect句でUDFを使用します。さらにデータを操作する前に、行がどのようなものかを確認するために [display](https://docs.databricks.com/user-guide/visualizations/index.html#display-function)関数を使用します。

In [0]:
df_signals = df_record_names.repartition('record_name').select(
  df_record_names.record_id,
  extract_signals_udf(df_record_names.record_name).alias('signal_info'))

df_signals = df_signals.select(df_signals.record_id,
                               df_signals.signal_info.patient_id.alias('patient_id'),
                               df_signals.signal_info.comments.alias('comments'),
                               df_signals.signal_info.signals.alias('signals'))

# the signal arrays are too large to display, so we'll drop them first
display(df_signals.drop('signals'))

record_id,patient_id,comments
patient042-s0140lre,patient042,"Map(Ventriculography -> Akinesia apex, Hypokinesia anterior wall, In hospital medication -> Metprolol Captopril, Left coronary artery stenoses (RIVA) -> RIVA proximal 25-50%, Pulmonary artery pressure (laod) (mean) -> 40 cmH2O, Peripheral blood Pressure (syst/diast) -> 120/80 mmHg, Medication after discharge -> Isosorbit-Dinitrate Metprolol Captopril ASA, Therapy -> , Cardiac index (load) -> 5,8 l/min/sqrmBSA, Cardiac output (at rest) -> 5,62 l/min, Previous infarction (2) date -> n/a, Hemodynamics -> , Pulmonary capillary wedge pressure (load) -> 31 cmH2O, Pulmonary artery pressure (at rest) (syst/diast) -> 23/10 cmH2O, Cardiac output (load) -> 11,85 l/min, Acute infarction (localization) -> antero-septal, Right coronary artery stenoses (RCA) -> No stenoses, Number of coronary vessels involved -> 1, age -> 49, ECG date -> 11/03/1991, sex -> male, Medication pre admission -> ASA, Pulmonary artery pressure (at rest) (mean) -> 16 cmH2O, Stroke volume index (load) -> 40 ml/beat, Pulmonary capillary wedge pressure (at rest) -> 11 cmH2O, Aorta (at rest) (syst/diast) -> n/a, Lytic agent -> Streptokinase, Dosage (lytic agent) -> 1.5 Mio IE, Cardiac index (at rest) -> 2,7 l/min/sqrmBSA, Left ventricular enddiastolic pressure -> n/a, Catheterization date -> 14-Aug-91, Diagnose -> , Left coronary artery stenoses (RCX) -> No stenoses, Smoker -> yes, Pulmonary artery pressure (laod) (syst/diast) -> 49/23 cmH2O, Former infarction (localization) -> no, Previous infarction (1) date -> n/a, Echocardiography -> n/a, Additional medication -> ASA Ca-antagonist Isosorbit-Dinitrate Molsidomin Metoprolol, Additional diagnoses -> no, Reason for admission -> Myocardial infarction, Aorta (at rest) mean -> n/a, Chest X-ray -> normal, Stroke volume index (at rest) -> 35.0 ml/beat, Start lysis therapy (hh.mm) -> 01, Admission date -> 26-Feb-91, Infarction date (acute) -> 26-Feb-91, Infarction date -> 26-Feb-91)"
patient016-s0076lre,patient016,"Map(Ventriculography -> normal, In hospital medication -> Isosorbit-Dinitrate ASA Lovastatin Metprolol Ranitidin, Left coronary artery stenoses (RIVA) -> No stenoses, Pulmonary artery pressure (laod) (mean) -> 12 cmH2O, Peripheral blood Pressure (syst/diast) -> 120/70 mmHg, Medication after discharge -> Isosorbit-Dinitrate Metoprolol Lovastatin Ranitidin, Therapy -> , Cardiac index (load) -> 4,65 l/min/sqrmBSA, Cardiac output (at rest) -> n/a, Previous infarction (2) date -> n/a, Hemodynamics -> , Pulmonary capillary wedge pressure (load) -> 1 cmH2O, Pulmonary artery pressure (at rest) (syst/diast) -> 11/2 cmH2O, Cardiac output (load) -> 9,59 l/min, Acute infarction (localization) -> infero-lateral, Right coronary artery stenoses (RCA) -> RCA proximal to ramus ventricularis dexter 70%, Number of coronary vessels involved -> 2, age -> 63, ECG date -> 17/12/1990, sex -> male, Medication pre admission -> Purin-antagonist Clofibrate Acipimox, Pulmonary artery pressure (at rest) (mean) -> 6 cmH2O, Stroke volume index (load) -> 38,4 ml/beat, Pulmonary capillary wedge pressure (at rest) -> 2 cmH2O, Aorta (at rest) (syst/diast) -> n/a, Lytic agent -> Urokinase, Dosage (lytic agent) -> 3 Mio IE, Cardiac index (at rest) -> 1,85 l/min/sqrmBSA, Left ventricular enddiastolic pressure -> n/a, Catheterization date -> 22-Nov-90, Diagnose -> , Left coronary artery stenoses (RCX) -> RCX close to apex 50%, Smoker -> no, Pulmonary artery pressure (laod) (syst/diast) -> 19/7 cmH2O, Former infarction (localization) -> no, Previous infarction (1) date -> n/a, Echocardiography -> n/a, Additional medication -> Heparin Isosorbit-Mononitrate ASA Maalox, Additional diagnoses -> Diabetes mellitus, Hyperuricemia, Reason for admission -> Myocardial infarction, Aorta (at rest) mean -> n/a, Chest X-ray -> normal, Stroke volume index (at rest) -> 23.8 ml/beat, Start lysis therapy (hh.mm) -> 18, Admission date -> 13-Nov-90, Infarction date (acute) -> 13-Nov-90, Infarction date -> 13-Nov-9

## 波形データをDelta Lakeに書き込む

今回の作業の難しい部分は終わりました！以下のセルではデータをDelta Lakeテーブルに保存しています。

In [0]:
df_signals.write.format("delta").mode("overwrite").save(DELTA_PATH)

# END